In [1]:
import pandas as pd
import simpy

In [2]:
base_dados = pd.read_csv("DoaçãoRim.csv", sep=",")

display(base_dados)

,Paciente Id,Data de chegada na fila,date,age_at_list_registration,age_cat,time_on_Dialysis,Prioridade,number_prior_transplant,patient_still_on_list,date_acutal,...,Transplant,Transplant_Y_N,X36MthsTx,Time_Tx,number_removed_list,removed_list,razon_removed,time,event,saida da fila sem transplante
0,1.0,1.0,03/01/2000,52,18.a.60,NaN,1.0,0,Não,43307,...,Sim,Sim,0,493,0.0,Não,NaN,1479,1,NaN
1,2.0,2.0,04/01/2000,51,18.a.60,0.0,0.0,1,Não,43307,...,Não,Não,0,611,1.0,Sim,Removido (suspenso > 365 dias),1833,3,NaN
2,3.0,2.0,04/01/2000,44,18.a.60,24.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,2583,0.0,Não,NaN,775,2,NaN
3,4.0,3.0,05/01/2000,53,18.a.60,6.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,6293,0.0,Não,NaN,1888,2,NaN
4,5.0,4.0,06/01/2000,61,Maior.60,2.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,4087,0.0,Não,NaN,1226,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48148,NaN,NaN,29/12/2017,50,18.a.60,5.0,NaN,0,Não,43307,...,Sim,Sim,1,197,NaN,Não,NaN,59,1,NaN
48149,NaN,NaN,29/12/2017,59,18.a.60,5.0,NaN,0,Sim,43307,...,Não,Não,0,697,NaN,Não,NaN,209,0,NaN
48150,NaN,NaN,29/12/2017,68,Maior.60,11.0,NaN,0,Sim,43307,...,Não,Não,0,697,NaN,Não,NaN,209,0,NaN
48151,NaN,NaN,29/12/2017,56,18.a.60,6.0,NaN,0,Sim,43307,...,Não,Não,0,697,NaN,Não,NaN,209,0,NaN


## Mantendo apenas os dados que serão utilizados 

In [3]:
remover = base_dados.drop(columns=["age_cat","time_on_Dialysis","date_acutal","Time_death","Transplant_Y_N","X36MthsTx","Time_Tx","event","razon_removed"])

remover.head()

,Paciente Id,Data de chegada na fila,date,age_at_list_registration,Prioridade,number_prior_transplant,patient_still_on_list,death,Sem transplante,Transplant,number_removed_list,removed_list,time,saida da fila sem transplante
0,1.0,1.0,03/01/2000,52,1.0,0,Não,Não,1.0,Sim,0.0,Não,1479,NaN
1,2.0,2.0,04/01/2000,51,0.0,1,Não,Não,0.0,Não,1.0,Sim,1833,NaN
2,3.0,2.0,04/01/2000,44,1.0,0,Não,Sim,0.0,Óbito Lista,0.0,Não,775,NaN
3,4.0,3.0,05/01/2000,53,1.0,0,Não,Sim,0.0,Óbito Lista,0.0,Não,1888,NaN
4,5.0,4.0,06/01/2000,61,1.0,0,Não,Sim,0.0,Óbito Lista,0.0,Não,1226,NaN


In [4]:
dataChegada = base_dados['Data de chegada na fila']
idPaciente = base_dados['Paciente Id']
tempo = base_dados['time']
prioridade = base_dados['Prioridade']
transplanteOcorreu = base_dados['Sem transplante']
def passandoLista(coluna):
    lista = []

    for item in coluna:
        lista.append(int(item))
        
    return lista

dataChegada100 = dataChegada[:100]
idPaciente100 = idPaciente[:100]
tempo100 = tempo[:100]
prioridade100 = prioridade[:100]
transplanteOcorreu100 = transplanteOcorreu[:100]
passandoLista(dataChegada100)


[1,
 2,
 2,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 6,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15]

In [5]:
passandoLista(idPaciente100)
passandoLista(tempo100)
passandoLista(prioridade100)

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [6]:
base_dados.head(n=60)

,Paciente Id,Data de chegada na fila,date,age_at_list_registration,age_cat,time_on_Dialysis,Prioridade,number_prior_transplant,patient_still_on_list,date_acutal,...,Transplant,Transplant_Y_N,X36MthsTx,Time_Tx,number_removed_list,removed_list,razon_removed,time,event,saida da fila sem transplante
0,1.0,1.0,03/01/2000,52,18.a.60,NaN,1.0,0,Não,43307,...,Sim,Sim,0,493,0.0,Não,NaN,1479,1,NaN
1,2.0,2.0,04/01/2000,51,18.a.60,0.0,0.0,1,Não,43307,...,Não,Não,0,611,1.0,Sim,Removido (suspenso > 365 dias),1833,3,NaN
2,3.0,2.0,04/01/2000,44,18.a.60,24.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,2583,0.0,Não,NaN,775,2,NaN
3,4.0,3.0,05/01/2000,53,18.a.60,6.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,6293,0.0,Não,NaN,1888,2,NaN
4,5.0,4.0,06/01/2000,61,Maior.60,2.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,4087,0.0,Não,NaN,1226,2,NaN
5,6.0,4.0,06/01/2000,24,18.a.60,6.0,1.0,0,Não,43307,...,Óbito Lista,Não,0,67,1.0,Sim,Removido (suspenso > 365 dias),5329,2,NaN
6,7.0,4.0,06/01/2000,27,18.a.60,8.0,1.0,0,Não,43307,...,Sim,Sim,0,547,0.0,Não,NaN,1641,1,NaN
7,8.0,4.0,06/01/2000,32,18.a.60,5.0,1.0,0,Não,43307,...,Sim,Sim,0,517,0.0,Não,NaN,1551,1,NaN
8,9.0,4.0,06/01/2000,30,18.a.60,4.0,1.0,0,Não,43307,...,Sim,Sim,0,10917,0.0,Não,NaN,3275,1,NaN
9,10.0,4.0,06/01/2000,37,18.a.60,3.0,1.0,0,Sim,43307,...,Não,Não,0,22587,0.0,Não,NaN,6776,0,NaN


In [11]:
#Simulação com tempo médio para transplante

def fila(env, recurso, paciente, chegada, saiuFila, prioridade, saida):
    yield env.timeout(chegada)

    with recurso.request(priority=prioridade) as req:
        yield req
        print("Paciente:", paciente, 'entrou na fila em', env.now)
        yield env.timeout(saida)
        if saiuFila:
            print("Paciente:", paciente, "saiu (morte ou desistência) da fila em", env.now)
        else:    
            print("Paciente:", paciente, 'foi transplantado em', env.now)

            # for index, item in enumerate(saiuFila_list):
            #     if item == 0:
            #         print("Paciente:", paciente, "saiu (morte ou desistência) da fila em", saida)
            #         saiuFila_list[index] = 1
            #         break
            # else:
            #     yield env.timeout(1745)
            #     print(paciente, 'saiu foi transplantado em', env.now)

env = simpy.Environment()
recurso = simpy.PriorityResource(env, capacity=7) # substituir por 4828/12 == 403
for i in range(100):
    env.process(fila(env, recurso, idPaciente100[i], dataChegada100[i], transplanteOcorreu100[i], prioridade100[i], 30))


# env.process(fila(env, recurso, 'Maria', 0, 0, 0))
# env.process(fila(env, recurso, 'Paula', 2, 2, 1))
# env.process(fila(env, recurso, 'Paulo', 6, 4, 1))
# env.process(fila(env, recurso, 'Alana', 0, 0, 1))
# env.process(fila(env, recurso, 'Sandra', 0, 0, 0))

env.run()

Paciente: 1.0 entrou na fila em 1.0
Paciente: 2.0 entrou na fila em 2.0
Paciente: 3.0 entrou na fila em 2.0
Paciente: 4.0 entrou na fila em 3.0
Paciente: 5.0 entrou na fila em 4.0
Paciente: 6.0 entrou na fila em 4.0
Paciente: 7.0 entrou na fila em 4.0
Paciente: 1.0 saiu (morte ou desistência) da fila em 31.0
Paciente: 8.0 entrou na fila em 31.0
Paciente: 2.0 foi transplantado em 32.0
Paciente: 3.0 foi transplantado em 32.0
Paciente: 9.0 entrou na fila em 32.0
Paciente: 10.0 entrou na fila em 32.0
Paciente: 4.0 foi transplantado em 33.0
Paciente: 11.0 entrou na fila em 33.0
Paciente: 5.0 foi transplantado em 34.0
Paciente: 6.0 foi transplantado em 34.0
Paciente: 7.0 saiu (morte ou desistência) da fila em 34.0
Paciente: 12.0 entrou na fila em 34.0
Paciente: 13.0 entrou na fila em 34.0
Paciente: 14.0 entrou na fila em 34.0
Paciente: 8.0 saiu (morte ou desistência) da fila em 61.0
Paciente: 15.0 entrou na fila em 61.0
Paciente: 9.0 saiu (morte ou desistência) da fila em 62.0
Paciente: 10.0

In [8]:
#Simulação da fila com as entradas e saídas da base de dados

def fila(env, recurso, paciente, chegada, saiuFila, prioridade, saida):
    yield env.timeout(chegada)
    saiuFila_list = [saiuFila]
   
    with recurso.request(priority=prioridade) as req:
        yield req
        print("Paciente:", paciente, 'atendida em', env.now)
        for index, item in enumerate(saiuFila_list):
            if item == 0:
                print("Paciente:", paciente, "saiu da fila em", saida)
                saiuFila_list[index] = 1
                break
        else:
            yield env.timeout(1745)
            print(paciente, 'saiu do atendimento em', env.now)

env = simpy.Environment()
recurso = simpy.PriorityResource(env, capacity=20)
for i in range(100):
    env.process(fila(env, recurso, idPaciente100[i], dataChegada100[i], transplanteOcorreu100[i], prioridade100[i], tempo100[i]))

env.run()

Paciente: 1.0 atendida em 1.0
Paciente: 2.0 atendida em 2.0
Paciente: 2.0 saiu da fila em 1833
Paciente: 3.0 atendida em 2.0
Paciente: 3.0 saiu da fila em 775
Paciente: 4.0 atendida em 3.0
Paciente: 4.0 saiu da fila em 1888
Paciente: 5.0 atendida em 4.0
Paciente: 5.0 saiu da fila em 1226
Paciente: 6.0 atendida em 4.0
Paciente: 6.0 saiu da fila em 5329
Paciente: 7.0 atendida em 4.0
Paciente: 8.0 atendida em 4.0
Paciente: 9.0 atendida em 4.0
Paciente: 10.0 atendida em 4.0
Paciente: 10.0 saiu da fila em 6776
Paciente: 11.0 atendida em 4.0
Paciente: 11.0 saiu da fila em 6434
Paciente: 12.0 atendida em 4.0
Paciente: 12.0 saiu da fila em 1831
Paciente: 13.0 atendida em 4.0
Paciente: 13.0 saiu da fila em 1904
Paciente: 14.0 atendida em 4.0
Paciente: 15.0 atendida em 4.0
Paciente: 15.0 saiu da fila em 1301
Paciente: 16.0 atendida em 4.0
Paciente: 17.0 atendida em 4.0
Paciente: 17.0 saiu da fila em 4436
Paciente: 18.0 atendida em 4.0
Paciente: 18.0 saiu da fila em 6006
Paciente: 19.0 atendida e